In [ ]:
from tqdm import tqdm
from gpt_setting import *
import json
import utils

utils.temperature = 1
utils.delay_time = 2
language = "en"

def rephrase():
    with open('dataset/questionnaires.json', 'r') as dataset:
        data = json.load(dataset)
    statements = data["BFI"]["questions"][language]["v1"]["statements"].items()
    existed_statements = [statement[1] for statement in data["BFI"]["questions"][language].items() if statement[0].startswith('v')]
    
    rephrased = []
    for count, statement in tqdm(enumerate(statements)):
        existed_rephrased_statements = [s["statements"][str(count+1)] for s in existed_statements]
        existed_rephrased_str = '"' + '", "'.join(existed_rephrased_statements) + '"'
        while True:
            with open(f'prompt_template/{language}/rephrase_{language}.txt', 'r') as file:
                _, prompt = file.read().strip().split("<commentblockmarker>###</commentblockmarker>")
            prompt = prompt.replace('!<INPUT 0>!', statement[1])
            prompt = prompt.replace('!<INPUT 1>!', existed_rephrased_str)
            inputs = [
                {"role": "system", "content": data["BFI"]["questions"][language]["system_prompt"]},
                {"role": "user", "content": prompt}
            ]
            print(inputs)
            try:
                response = chat('gpt-4', inputs).strip()
                print(response)
                parsered_responses = json.loads(response)
                parsered_responses = parsered_responses["sentence"]
                break
            except:
                pass
        rephrased.append(parsered_responses)
        with open('rephrased_q3.txt', 'a') as f:
            f.write(f'{parsered_responses}\n')
        
    return

rephrase()
